In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Flatten

from sklearn.preprocessing import OneHotEncoder

# Load & Preprocess Data

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
label_dict = {
  0: 'T-shirt/top',
  1: 'Trouser',
  2: 'Pullover',
  3: 'Dress',
  4: 'Coat',
  5: 'Sandal',
  6: 'Shirt',
  7: 'Sneaker',
  8: 'Bag',
  9: 'Ankle boot'
}

## Normalise Image Data

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

## One-Hot Encode Labels

In [ ]:
ohe = OneHotEncoder(categories='auto')
ohe.fit(y_train.reshape(-1, 1))
y_train = ohe.transform(y_train.reshape(-1, 1))
y_test = ohe.transform(y_test.reshape(-1, 1))

# Basic Exploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print('train data shape: {}; target shape: {}'.format(X_train.shape, y_train.shape))
print('test data shape: {}; target shape: {}'.format(X_test.shape, y_test.shape))

In [ ]:
sample = X_train[0]
plt.imshow(sample, cmap='gray')

In [ ]:
nrows = 4
ncols = 4

fig, axes = plt.subplots(nrows, ncols)
fig.set_size_inches(16, 16)

i = 0

for row in range(nrows):
    for col in range(ncols):
        img = X_train[i]
        label = label_dict[y_train[i].argmax()]

        axis = axes[row, col]
        axis.imshow(img, cmap='gray')
        axis.get_xaxis().set_visible(False)
        axis.get_yaxis().set_visible(False)
        axis.set_title(label)
        
        i += 1

# Building the Model

In [ ]:
batch_size = 32
img_dim = X_train[0].shape

print('Image Dimensions: {}'.format(img_dim))

In [ ]:
model = keras.models.Sequential([
    Flatten(input_shape=img_dim),
    Dense(512),
    Activation('relu'),
    Dense(10),
    Activation('softmax')
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training and Evaluating the Model

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=10)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy:', test_acc)

In [ ]:
nrows = 6
ncols = 6
ntotal = nrows * ncols
i = 0

fig, axes = plt.subplots(nrows, ncols)
fig.set_size_inches(20, 16)

predictions = model.predict_proba(X_test[:ntotal])

for row in range(nrows):
    for col in range(ncols):
        img = X_test[i]        
        predicted_label = label_dict[predictions[i].argmax()]
        probability = predictions[i].max() * 100
        actual_label = label_dict[y_test[i].argmax()]
        title_colour = 'green' if predicted_label == actual_label else 'red'

        axis = axes[row, col]
        axis.imshow(img, cmap='gray')
        axis.get_xaxis().set_visible(False)
        axis.get_yaxis().set_visible(False)
        axis.set_title('{} {:.2f}% ({})'.format(predicted_label, probability, actual_label), color=title_colour)
        
        i += 1

# CNN

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout
import numpy as np

## Expand Data Dimensions

Add in an extra dimension for channels.
The images in the Fashion MNIST dataset are grayscale so there is only one channel.

In [ ]:
print(img_dim)
print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train_with_channels = np.expand_dims(X_train, axis=3)
X_test_with_channels = np.expand_dims(X_test, axis=3)
cnn_input_shape = X_train_with_channels[0].shape

print(img_dim)
print(X_train_with_channels.shape)
print(X_test_with_channels.shape)

## CNN Model

In [ ]:
cnn = keras.models.Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=cnn_input_shape),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    
    Dense(10, activation='softmax')
])

In [ ]:
cnn.summary()

In [ ]:
cnn.compile(optimizer='rmsprop',
           loss=keras.losses.categorical_crossentropy,
           metrics=['accuracy'])

## Train & Evaluate Model

In [ ]:
history = cnn.fit(X_train_with_channels, y_train, 
                  validation_data=(X_test_with_channels, y_test), 
                  batch_size=batch_size, 
                  epochs=10)

In [ ]:
plt.figure(figsize=[8,6])
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
%%time

from sklearn.metrics import confusion_matrix, classification_report

predictions = cnn.predict(X_test_with_channels)
predictions = predictions.argmax(axis=1)
ground_truth = y_test.argmax(axis=1)

print(confusion_matrix(ground_truth, predictions))
print(classification_report(ground_truth, predictions))

In [ ]:
nrows = 6
ncols = 6
ntotal = nrows * ncols
i = 0

fig, axes = plt.subplots(nrows, ncols)
fig.set_size_inches(20, 16)

predictions = cnn.predict_proba(X_test_with_channels[:ntotal])

for row in range(nrows):
    for col in range(ncols):
        img = X_test[i]        
        predicted_label = label_dict[predictions[i].argmax()]
        probability = predictions[i].max() * 100
        actual_label = label_dict[y_test[i].argmax()]
        title_colour = 'green' if predicted_label == actual_label else 'red'

        axis = axes[row, col]
        axis.imshow(img, cmap='gray')
        axis.get_xaxis().set_visible(False)
        axis.get_yaxis().set_visible(False)
        axis.set_title('{} {:.2f}% ({})'.format(predicted_label, probability, actual_label), color=title_colour)
        
        i += 1